Importando arquivos

In [50]:
import pandas as pd
base_train = pd.read_csv('distribuicao-renda.csv', encoding='utf-8', sep=';')

Transformando os valores categóricos

In [51]:
from sklearn.preprocessing import MinMaxScaler

# Deixando apenas dados de estado
base_train = base_train.loc[base_train['Ente Federativo'] != 'BRASIL']

base_train = base_train.drop(columns=['Ente Federativo', 'Ano-calendário', 'Centil'], axis=1)

# Formatando valores para um valor monetário internacional
for column in base_train.columns:
    if base_train[column].dtype == 'object':
        base_train[column] = base_train[column].str.replace('.','')
        base_train[column] = base_train[column].str.replace(',','.')
        base_train[column] = base_train[column].astype(float)

# Preenchendo valores vazios com 0
base_train.fillna(0,inplace=True)

# Inicializar o scaler
scaler = MinMaxScaler()

columns_transformed = base_train.columns
base_train[columns_transformed] = scaler.fit_transform(base_train[columns_transformed])

base_train.columns

Index(['Quantidade de Contribuintes',
       'Rendimentos Tributaveis - Limite Superior da RTB do Centil [R$ milhões]',
       'Rendimentos Tributaveis - Soma da RTB do Centil [R$ milhões]',
       'Rendimentos Tributaveis - RTB Acumulada do Centil [R$ milhões]',
       'Rendimentos Tributaveis - Média da RTB do Centil [R$]',
       'Rendimentos Sujeitos à Tribut. Exclusiva [R$ milhões]',
       'Rendimentos Isentos - Lucros e dividendos [R$ milhões]',
       'Rendimentos Isentos - Rendim. Sócio/Titular ME/EPP Opt SIMPLES [R$ milhões]',
       'Rendimentos Isentos - Outros Rendimentos Isentos [R$ milhões]',
       'Despesas Dedutíveis - Previdência [R$ milhões]',
       'Despesas Dedutíveis - Dependentes [R$ milhões]',
       'Despesas Dedutíveis - Instrução [R$ milhões]',
       'Despesas Dedutíveis - Médicas [R$ milhões]',
       'Despesas Dedutíveis - Pensão Alimentícia [R$ milhões]',
       'Despesas Dedutíveis - Livro-Caixa [R$ milhões]',
       'Imposto Devido [R$ milhões]', 'Ben

In [52]:
base_train.describe()

,Quantidade de Contribuintes,Rendimentos Tributaveis - Limite Superior da RTB do Centil [R$ milhões],Rendimentos Tributaveis - Soma da RTB do Centil [R$ milhões],Rendimentos Tributaveis - RTB Acumulada do Centil [R$ milhões],Rendimentos Tributaveis - Média da RTB do Centil [R$],Rendimentos Sujeitos à Tribut. Exclusiva [R$ milhões],Rendimentos Isentos - Lucros e dividendos [R$ milhões],Rendimentos Isentos - Rendim. Sócio/Titular ME/EPP Opt SIMPLES [R$ milhões],Rendimentos Isentos - Outros Rendimentos Isentos [R$ milhões],Despesas Dedutíveis - Previdência [R$ milhões],...,Despesas Dedutíveis - Instrução [R$ milhões],Despesas Dedutíveis - Médicas [R$ milhões],Despesas Dedutíveis - Pensão Alimentícia [R$ milhões],Despesas Dedutíveis - Livro-Caixa [R$ milhões],Imposto Devido [R$ milhões],Bens e Direitos - Imóveis [R$ milhões],Bens e Direitos - Móveis [R$ milhões],Bens e Direitos - Financeiros [R$ milhões],Bens e Direitos - Outros Bens e Direitos [R$ milhões],Dívidas e Ônus [R$ milhões]
count,44550.000000,44550.000000,44550.000000,44550.000000,44550.000000,44550.000000,44550.000000,44550.000000,44550.000000,44550.000000,...,44550.000000,44550.000000,44550.000000,44550.000000,44550.000000,44550.000000,44550.000000,44550.000000,44550.000000,44550.000000
mean,0.090901,0.001098,0.005880,0.017564,0.028039,0.002262,0.003688,0.004719,0.003620,0.006180,...,0.027450,0.009247,0.008941,0.001842,0.002724,0.005536,0.004725,0.002092,0.000708,0.002125
std,0.200725,0.013421,0.019875,0.049146,0.050822,0.014278,0.017286,0.019815,0.016038,0.022033,...,0.068992,0.029869,0.029533,0.017205,0.017053,0.019800,0.014488,0.014310,0.007863,0.013542
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.001877,0.000058,0.000497,0.000720,0.006478,0.000083,0.000122,0.000110,0.000222,0.000209,...,0.000787,0.000213,0.000170,0.000003,0.000000,0.000311,0.000393,0.000064,0.000017,0.000095
50%,0.005262,0.000099,0.001508,0.003747,0.011186,0.000307,0.000559,0.000583,0.000734,0.001157,...,0.005511,0.001584,0.001425,0.000040,0.000057,0.001029,0.001180,0.000242,0.000066,0.000355
75%,0.024013,0.000210,0.004551,0.013640,0.023464,0.001165,0.002303,0.002689,0.002511,0.004399,...,0.024268,0.006858,0.006272,0.000373,0.000881,0.004063,0.003925,0.000989,0.000280,0.001377
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [53]:
correlation_values = base_train.corr().sort_values(by='Imposto Devido [R$ milhões]', ascending=False)
print(correlation_values[['Imposto Devido [R$ milhões]']])

                                                    Imposto Devido [R$ milhões]
Imposto Devido [R$ milhões]                                            1.000000
Rendimentos Tributaveis - Soma da RTB do Centil...                     0.944055
Despesas Dedutíveis - Previdência [R$ milhões]                         0.926311
Rendimentos Sujeitos à Tribut. Exclusiva [R$ mi...                     0.922522
Bens e Direitos - Financeiros [R$ milhões]                             0.916928
Despesas Dedutíveis - Pensão Alimentícia [R$ mi...                     0.905580
Bens e Direitos - Imóveis [R$ milhões]                                 0.899707
Rendimentos Isentos - Lucros e dividendos [R$ m...                     0.863753
Despesas Dedutíveis - Livro-Caixa [R$ milhões]                         0.860338
Rendimentos Isentos - Outros Rendimentos Isento...                     0.858919
Despesas Dedutíveis - Médicas [R$ milhões]                             0.809949
Bens e Direitos - Móveis [R$ milhões]   

In [54]:
from sklearn.model_selection import train_test_split

feature_columns = base_train.drop( columns=['Imposto Devido [R$ milhões]'], axis=1).columns

target = base_train['Imposto Devido [R$ milhões]']

features = base_train[feature_columns]
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [55]:
from sklearn.linear_model import LinearRegression
import numpy as np

model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [56]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

train_predictions = model.predict(X_train)
test_predictions = model.predict(X_test)

mae_train = mean_absolute_error(y_train, train_predictions)
mae_test = mean_absolute_error(y_test, test_predictions)
rmse_train = np.sqrt(mean_squared_error(y_train, train_predictions))
rmse_test = np.sqrt(mean_squared_error(y_test, test_predictions))
mse_train = mean_squared_error(y_train, train_predictions)
mse_test = mean_squared_error(y_train, train_predictions)

print("Score Treinamento: ", model.score(X_train, y_train))
print("Score Teste: ", model.score(X_test, y_test))
print("MSE Treinamento: ",mse_train)
print("MSE Teste: ", mse_test)
print("Erro Quadrático Médio (RMSE) do treinamento:", rmse_train)
print("Erro Quadrático Médio (RMSE) do teste:", rmse_test)
print("Erro Absoluto Médio (MAE) do treinamento:", mae_train)
print("Erro Absoluto Médio (MAE) do teste:", mae_test)

Score Treinamento:  0.9943380532251551
Score Teste:  0.9885649560509402
MSE Treinamento:  1.8464796116382066e-06
MSE Teste:  1.8464796116382066e-06
Erro Quadrático Médio (RMSE) do treinamento: 0.0013588523141380032
Erro Quadrático Médio (RMSE) do teste: 0.0013072251340391688
Erro Absoluto Médio (MAE) do treinamento: 0.0005991274673102892
Erro Absoluto Médio (MAE) do teste: 0.0005898698744325214
